# Tibero DB 접속 환경설정

In [12]:
DSNNAME = "Tibero6"
DBUSER =""
DBPWD = ""


In [17]:
import pyodbc
import codecs
import pandas as pd

from urllib.request import urlopen
from bs4 import BeautifulSoup # document parser
import xml.etree.ElementTree as elemTree
import time

import socket

from datetime import datetime

now = datetime.now()

# Tibero DB CONNECT, INSERT,  SELECT
def db_connect():
    try: 
        connection_string = 'DSN=Tibero6;UID=ariadm;PWD=tmxkqjrs'
        conn = pyodbc.connect(connection_string)
    except BaseException:
        print("DB connection Error") 
    cursor = conn.cursor()  
    
    return conn, cursor

# 도시 아파트 코드 API 호출
def db_insert_batch(sql, APT_CODE, APT_NAME, SIDO_CODE, CRT_DTHR, CRT_IP_MAC):
    conn, cursor = db_connect()
    with conn: # with 절 종료 시점에 db 닫음
        with conn.cursor() as cur:
            cur.execute(sql, (APT_CODE, APT_NAME, SIDO_CODE, CRT_DTHR, CRT_IP_MAC))

# 아파트 기본정보 API 호출
def db_update_batch(sql, apt_code, addr, household_cnt, is_addr_error, is_lon_error):
    conn, cursor = db_connect()
    with conn: # with 절 종료 시점에 db 닫음
        with conn.cursor() as cur:
            cur.execute(sql, (apt_code, addr, household_cnt, is_addr_error, is_lon_error))

# 결과
def db_select(sql):
    conn, cursor = db_connect()
    cursor.execute(sql)
    result = pd.DataFrame(cursor.fetchall())   
    conn.close()
    
    return result

# open api를 호출해 전국 아파트 목록 xml 데이터를 획득하는 함수입니다
def xml_total_Apt(serviceUrl, pageNo, numOfRows, serviceKey='tP%2FocKCOSN8ZJt07XRz8uFOK4o%2BRsIooWIXfLbbldmsSIoPg11G551gCTpQIUtwAa5yDr1%2BxtQfANq27DNkxKA%3D%3D'): 
    max_retries = 4
    retry_delay = 10  # seconds
 
    url = serviceUrl + '?pageNo=' + pageNo + '&numOfRows=' + numOfRows + '&serviceKey=' + serviceKey           
    
    for attempt in range(max_retries):
        try:
            result = urlopen(url, timeout=5)  # request 요청 
            xmlobj = BeautifulSoup(result, 'lxml-xml') # xml 데이터 획득
            
            break  # Exit the loop if the request succeeds
            
        except (urllib.error.URLError):
            
            if attempt < max_retries - 1:
                print(f'Retrying in {retry_delay} seconds...')
                time.sleep(retry_delay)
            else:
                print('Maximum retries exceeded')    
    
    return xmlobj


# open api를 호출해 도시의 아파트 목록 xml 데이터를 획득하는 함수입니다
def xml_sido_Apt(serviceUrl, sidoCode, pageNo, numOfRows, serviceKey='tP%2FocKCOSN8ZJt07XRz8uFOK4o%2BRsIooWIXfLbbldmsSIoPg11G551gCTpQIUtwAa5yDr1%2BxtQfANq27DNkxKA%3D%3D'): 
    max_retries = 4
    retry_delay = 10  # seconds
 
    url = serviceUrl + '?sidoCode=' + sidoCode + '&pageNo=' + pageNo + '&numOfRows=' + numOfRows + '&serviceKey=' + serviceKey           
    
    for attempt in range(max_retries):
        try:
            result = urlopen(url, timeout=5)  # request 요청 
            xmlobj = BeautifulSoup(result, 'lxml-xml') # xml 데이터 획득
            
            break  # Exit the loop if the request succeeds
            
        except (urllib.error.URLError):
            
            if attempt < max_retries - 1:
                print(f'Retrying in {retry_delay} seconds...')
                time.sleep(retry_delay)
            else:
                print('Maximum retries exceeded')    
    
    return xmlobj



# 시스템에 접속 된 호스트 이름
print(socket.gethostbyname(socket.gethostname()))


# 현재 시간
print("문자열 변환 : ", now.strftime('%Y/%m/%d'))


211.219.7.36
문자열 변환 :  2023/02/17


# 공공데이터 오픈API 크롤링, Tibero로 저장

In [15]:
# 전국 아파트 단지 코드 조회
def setTotalAptList():
    
    sql = "INSERT INTO CMN_APT_BATCH (APT_CODE, APT_NAME, SIDO_CODE, CRT_DTHR, CRT_IP_MAC, DRT_MENU_ID, CRT_MENU_SEQ) VALUES(%s, %s, %s, %s, %s, 'Mymap', '20230208144520437681')"
    sql2 = "SELECT * FROM CMN_APT_BATCH"
    
    crt_host = socket.gethostbyname(socket.gethostname())
    crt_dthr = now.strftime('%Y/%m/%d')
    
    # xml 데이터 획득
    xmlDoc = xml_total_Apt('http://apis.data.go.kr/1613000/AptListService2//getTotalAptList','1','1')
   
    resultCode = xmlDoc.find('resultCode')  # 결과 상태 코드 확인
    
    if resultCode is not None: # 데이터가 있는 경우
        totalCount = xmlDoc.find('totalCount').get_text() # 전체 데이터 개수 확인
        numOfRows = xmlDoc.find('numOfRows').get_text()  # 한 페이지 당 출력 데이터 개수 확인

        totalCount = int(totalCount)
        numOfRows = int(numOfRows)

        loopCount = round(totalCount/numOfRows) # 총 open api 호출 횟수 소숫점 첫째자리에서 바로 반올림
        
        if ((loopCount * numOfRows) < totalCount): # 총 반복 횟수 보정 
            loopCount = loopCount + 1
        
        for pageNo in range(1, loopCount):  #  페이지 반복문
            # xml 데이터 획득
            xmlDoc = xml_total_Apt('http://apis.data.go.kr/1613000/AptListService2//getTotalAptList', str(pageNo),'10')
            print("현재 페이지 번호입니다 >>> ", pageNo)
            data = xmlDoc.findAll('item')  # xml에서 데이터 요소 찾기
            if data is not None: # 데이터가 있는 경우
                print(data)
                for item in data:
                    as1 = item.find('as1')
                    apt_code = item.find('kaptCode')
                    apt_name = item.find('kaptName')

                    if as1 is not None :
                        if "서울" in as1.get_text():
                            sido_code= "11"

                        elif "부산" in as1.get_text():
                            sido_code= "26"

                        elif "대구" in as1.get_text():
                            sido_code= "27"

                        elif "인천" in as1.get_text():
                            sido_code= "28"

                        elif "광주" in as1.get_text():
                            sido_code= "29"

                        elif "대전" in as1.get_text():
                            sido_code= "30"

                        elif "울산" in as1.get_text():
                            sido_code= "31"

                        elif "경기" in as1.get_text():
                            sido_code= "41"

                        elif "강원" in as1.get_text():
                            sido_code= "42"

                        elif "충청남도" in as1.get_text():
                            sido_code= "44"

                        elif "충청북도" in as1.get_text():
                            sido_code= "43"

                        elif "전라북도" in as1.get_text():
                            sido_code= "45"

                        elif "전라남도" in as1.get_text():
                            sido_code= "46"

                        elif "경상북도" in as1.get_text():
                            sido_code= "47"

                        elif "경상남도" in as1.get_text():
                            sido_code= "48"

                        elif "제주" in as1.get_text():
                            sido_code= "50"

                        else: 
                            sido_code="-1"

                    if apt_code is not None:
                        aptCode = apt_code.get_text()

                    if apt_name is not None:
                        aptName = apt_name.get_text()

                    print(aptCode, aptName, sido_code, crt_dthr, crt_host)
                    try:
                        db_insert_batch(sql, aptCode, aptName, sido_code, crt_dthr, crt_host)  # 저장

                        df= db_select(sql2) # 결과
                        print(df)
                    except Exception as e:           
                        print(e)               

            else: # 데이터가 없는 경우
                print("Data not found.")       
    
    return loopCount

In [16]:
setTotalAptList()

NameError: name 'urllib' is not defined

#### 시도코드로 단지 코드 조회

In [11]:
sido_query="SELECT CODE_USE1 FROM CMN_COMMON_CODE WHERE CODE_CLASS = 'CMN084' AND CODE_USE1 IS NOT NULL   AND CODE != 'CODEHEADER' ORDER BY PRT_ORD"

rows =  db_select(sido_query)
sido_cd =rows[0]
lst   = sido_cd.values.tolist()

# convert the list of tuples to a list of strings
sido_code = [tup[0] for tup in lst]
print(sido_code)
sido_code[0]

['11', '26', '27', '28', '29', '30', '31', '41', '42', '44', '45', '46', '47', '48', '50']


In [ ]:
# 시도코드로 단지 코드 조회
def getSidoAptList(sido_code):
    # open api를 호출해 xml 데이터 획득
    xmlDoc = xml_sido_Apt('http://apis.data.go.kr/1613000/AptListService2/getSidoAptList',sido_code,'1','10')
   
    resultCode = xmlDoc.find('resultCode')  # 결과 상태 코드 확인
    
    if resultCode is not None: # 데이터가 있는 경우
        totalCount = xmlDoc.find('totalCount').get_text() # 전체 데이터 개수 확인
        numOfRows = xmlDoc.find('numOfRows').get_text()  # 한 페이지 당 출력 데이터 개수 확인

        totalCount = int(totalCount)
        numOfRows = int(numOfRows)

        loopCount = round(totalCount/numOfRows) # 총 open api 호출 횟수 소숫점 첫째자리에서 바로 반올림

    if ( (loopCount * numOfRows) < totalCount):
        loopCount = loopCount + 1
    
    return loopCount